In [1]:
from google.colab import drive
drive.mount('/content/drive')
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
your_module = drive.CreateFile({'id':'174Ws0LnaEQdK7UmOKh-8GJwftO18Mfej'})
your_module.GetContentFile('metric.py')
your_module1 = drive.CreateFile({'id':'11ssSCDodnZ6_jXcHLZ8AqYHbg7g7K8Be'})
your_module1.GetContentFile('threshold.py')
your_module2 = drive.CreateFile({'id':'1JpuTOGEKMq9iZcO6Tp4ET9icd5EG0P0D'})
your_module2.GetContentFile('cat_list.py')
your_module2 = drive.CreateFile({'id':'1gsMrGL0BfVASQICf-eesMRelmCgMLTYs'})
your_module2.GetContentFile('cat_list_show.py')
from metric import *
from threshold import *
from cat_list import *
from cat_list_show import *

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import tensorflow as tf
import numpy as np
from keras import backend as K
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding, LSTM,Conv1D
from tensorflow.keras.optimizers import Adam,Adagrad,Nadam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dropout,GlobalMaxPooling1D,Conv1D,BatchNormalization
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras import initializers, regularizers, constraints
from sklearn.metrics import classification_report
from sklearn.metrics import fbeta_score
from sklearn.metrics import roc_auc_score, roc_curve,auc
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
def accuracy(y_true, y_pred):
    comp = K.equal(y_true, K.round(y_pred))
    return K.cast(K.all(comp, axis=-1), K.floatx())
def fbeta(y_true, y_pred, threshold_shift=0.005):
    beta = 2
    y_pred = K.clip(y_pred, 0, 1)
    y_pred_bin = K.round(y_pred + threshold_shift)
    tp = K.sum(K.round(y_true * y_pred_bin), axis=1) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)), axis=1)
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)), axis=1)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    beta_squared = beta ** 2
    return K.mean((beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon()))
def ffb (y_test, predictions, beta=1):
  pr = precision(y_test, predictions)
  re = recall(y_test, predictions)
  num = float((1+pow(beta,2))*pr*re)
  den = float(pow(beta,2)*pr + re)
  ffb = num/den
  return ffb
def accuracy3(y_test, predictions):
    accuracy = 0.0

    for i in range(y_test.shape[0]):
        intersection = 0.0
        union = 0.0
        for j in range(y_test.shape[1]):
            if int(y_test[i,j]) == 1 or int(predictions[i,j]) == 1:
                union += 1
            if int(y_test[i,j]) == 1 and int(predictions[i,j]) == 1:
                intersection += 1

            
        if union != 0:
            accuracy = accuracy + float(intersection/union)

    accuracy = float(accuracy/y_test.shape[0])

    return accuracy
#فراخوانی داده های آموزشی و آزمایشی
dataset = pd.read_excel('drive/My Drive/2020/c1.xlsx')
test_file = pd.read_excel('drive/My Drive/2020/test_cat_plo.xlsx')
S_comments_labels = dataset[['بازیگر#مثبت','بازیگر#منفی','بازیگری#مثبت','بازیگری#منفی','داستان#مثبت',
                              'داستان#منفی','دیالوگ#مثبت','دیالوگ#منفی','سبک#مثبت','سبک#منفی','فیلم#مثبت','فیلم#منفی',
                             'فیلم_نامه#مثبت','فیلم_نامه#منفی','فیلمبرداری#مثبت','فیلمبرداری#منفی',
                             'لوکیشن#مثبت','لوکیشن#منفی','محتوا#مثبت','محتوا#منفی','موسیقی#مثبت',
                             'موسیقی#منفی','موضوع#مثبت','موضوع#منفی','کارگردان#مثبت',
                             'کارگردان#منفی','گریم#مثبت','گریم#منفی']]

S_test_comments_labels = test_file[['بازیگر#مثبت','بازیگر#منفی','بازیگری#مثبت','بازیگری#منفی','داستان#مثبت',
                              'داستان#منفی','دیالوگ#مثبت','دیالوگ#منفی','سبک#مثبت','سبک#منفی','فیلم#مثبت','فیلم#منفی',
                             'فیلم_نامه#مثبت','فیلم_نامه#منفی','فیلمبرداری#مثبت','فیلمبرداری#منفی',
                             'لوکیشن#مثبت','لوکیشن#منفی','محتوا#مثبت','محتوا#منفی','موسیقی#مثبت',
                             'موسیقی#منفی','موضوع#مثبت','موضوع#منفی','کارگردان#مثبت',
                             'کارگردان#منفی','گریم#مثبت','گریم#منفی']]
category_list=['بازیگر#مثبت','بازیگر#منفی','بازیگری#مثبت','بازیگری#منفی','داستان#مثبت',
                              'داستان#منفی','دیالوگ#مثبت','دیالوگ#منفی','سبک#مثبت','سبک#منفی','فیلم#مثبت','فیلم#منفی',
                             'فیلم_نامه#مثبت','فیلم_نامه#منفی','فیلمبرداری#مثبت','فیلمبرداری#منفی',
                             'لوکیشن#مثبت','لوکیشن#منفی','محتوا#مثبت','محتوا#منفی','موسیقی#مثبت',
                             'موسیقی#منفی','موضوع#مثبت','موضوع#منفی','کارگردان#مثبت',
                             'کارگردان#منفی','گریم#مثبت','گریم#منفی']
stop_words =['به','شکل','ممکن','میشه','دلیل','رو','فقط','در','خانم','آقای','اگر','مریلا',
            'این','هم','اسامی','زارعی','بهرام','رادان','حیایی','پارسا','پیروزفر','و','صابر','ابر','حال','های',
            'ها','یه','ای','بابت','آبیار','از','تا','هفتاد','درصد','الناز','دو','اول','شاکردوست','سیمرغ','زن','تاریخ','شاکر','هاش','ریگی','اش','معادی','نرگس','دوم','ابیار','پیمان','اقای','بهداد','مهناز','حامد','عطاران','علی','نوید','ام','امیر','۱۰','محمدزاده','نفر','تیکت','سعید','لباس','خدا','هانیه'
             ,'توسلی','سحر','۲۰','ثقفی','رضا','کریمی','روسی','۴','کامنت','عبدالمالک','ش','اکبر','هیس','مهرجویی','طباطبایی','سرخپوست','عبدی','شش','شیطان','پرستی'
            ]
#پیش پردازش
import re
def preprocess_text(sentence):
    # Removing multiple spaces
    sentence = re.sub(r'@\S+', '', sentence)
    sentence = re.sub(r'!\S+', '', sentence)
    sentence = re.sub(r'؟\S+', '', sentence)
    sentence = re.sub(r'[.]', ' ', sentence)
    sentence = re.sub(r'[/]', '', sentence)
    sentence = re.sub(r'[،]', ' ', sentence)
    sentence = re.sub(r'[؛]', '', sentence)
    sentence = sentence.split()
    sentence =[word for word in sentence if word not in stop_words]
    sentence = ' '.join(sentence)
    return sentence
def ReplacetwoMore(s):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL) 
    return pattern.sub(r"\1", s)
dataset['clean_text'] = dataset['Sentence'].apply(ReplacetwoMore)
test_file['clean_text'] = test_file['Sentence'].apply(ReplacetwoMore)
X = []
sentences = list(dataset["clean_text"])
for sen in sentences:
    X.append(preprocess_text(sen))    
X_t = []
sentences2 = list(test_file["clean_text"])
for sen in sentences2:
    X_t.append(preprocess_text(sen))
y = S_comments_labels.values
y_test1=S_test_comments_labels.values
target =y
data = X
target1 = y_test1
data1 = X_t
x_train, x_test = data, data1
y_train, y_test = target, target1
#شمارش تعداد کلمات منحصر به فرد
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import FreqDist
all_words=' '.join(data)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
#شمارش بزرگترین طول متن
r_len=[]
for text in data:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)   
MAX_REVIEW_LEN=np.max(r_len)
num_words = num_unique_word
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(data)
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)
max_tokens = MAX_REVIEW_LEN
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,padding='post')
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,padding='post')
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text
x1=[]
x2=[]
x3=[]
x4=[]
x5=[]
x6=[]
x7=[]
x8=[]
x9=[]
x10=[]
x11=[]
x12=[]
x13=[]
x14=[]
x15=[]
x16=[]
x17=[]
x18=[]
x19=[]
x20=[]
x21=[]
x22=[]
x23=[]
x24=[]
x25=[]
x26=[]
x27=[]
x28=[]
x29=[]
x30=[]
x31=[]
x32=[]
x33=[]
x34=[]
x35=[]
x36=[]
x37=[]
x38=[]
x39=[]
x40=[]

dx1=[]
dx2=[]
dx3=[]
dx4=[]
dx5=[]
dx6=[]
dx7=[]
dx8=[]
dx9=[]
dx10=[]
dx11=[]
dx12=[]
dx13=[]
dx14=[]
dx15=[]
dx16=[]
dx17=[]
dx18=[]
dx19=[]
dx20=[]
dx21=[]
dx22=[]
dx23=[]
dx24=[]
dx25=[]
dx26=[]
dx27=[]
dx28=[]
dx29=[]
dx30=[]
dx31=[]
dx32=[]
dx33=[]
dx34=[]
dx35=[]
dx36=[]
dx37=[]
dx38=[]
dx39=[]
dx40=[]
km=1
j=1
i=0
from sklearn.model_selection import KFold
kfold = KFold(5, shuffle=False)
import time
st = time.time()
for i in range(10):
  for t, v in kfold.split(x_train_pad, y_train):
    start_time = time.time()
    embedding_size = 300
    model4 = tf.keras.Sequential()
    model4.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size,
                        input_length=max_tokens,
                        name='embedding_layer'))
    model4.add(Dropout(0.5))
    model4.add(GRU(units=256, return_sequences=True))
    model4.add(BatchNormalization())
    model4.add(GlobalMaxPooling1D())
    model4.add(Dense(200, activation='relu'))
    model4.add(Dropout(0.5))
    model4.add(Dense(28,activation='sigmoid'))
    optimizer =Nadam(learning_rate=1e-3)
    model4.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=[accuracy,fbeta])
    t_x = x_train_pad[t]
    val_x = x_train_pad[v]
    t_y = y_train[t]
    val_y = y_train[v]
    print("**********************************")
    print("Training on Fold: ",j," result:",km)
    j+=1
    print("**********************************")
    print("**********************************")
    history=model4.fit(t_x, t_y, epochs=20, batch_size=50,shuffle=False,verbose=0)
    p = model4.predict(x_test_pad)
    th(p,0.5)
    prr = model4.predict(x_test_pad)
    prr[prr>=0.5]=1
    prr[prr<0.5]=0
    ac=subsetAccuracy(y_test, p)
    hm=hammingLoss(y_test, p)
    acc=accuracy3(y_test, p)
    pm=precisionMacro(y_test, p)
    pr=precisionMicro(y_test, p)
    rm=recallMacro(y_test, p)
    rmm=recallMicro(y_test, p)
    fbm1=fbetaMacro(y_test, p, 1)
    fbmm1=fbetaMicro(y_test, p, 1)
    pmq=precision_score(y_test, prr, average='macro')
    prq=precision_score(y_test, prr, average='macro')
    fb=ffb(y_test, p, 1)
    fbq=ffb(y_test, prr, 1)
    aucc1=roc_auc_score(y_test, p,average='micro')
    aucc2=roc_auc_score(y_test, p,average='macro')     
    acq=subsetAccuracy(y_test, prr)
    hmq=hammingLoss(y_test, prr)
    accq=accuracy3(y_test, prr)



    am=accuracyMacro(y_test, p)
    amm=accuracyMicro(y_test, p)
    amq=accuracyMacro(y_test, prr)
    ammq=accuracyMicro(y_test, prr)
    pmq=precision_score(y_test, prr, average='macro')
    prq=precision_score(y_test, prr, average='micro')

    rmq=recall_score(y_test, prr, average='macro')
    rmmq=recall_score(y_test, prr, average='micro')
    fbm=fbeta_score(y_test, prr, average='macro', beta=1)
    fbmm=fbeta_score(y_test, prr, average='micro', beta=1)
    aucc11=roc_auc_score(y_test, prr,average='micro')
    aucc22=roc_auc_score(y_test, prr,average='macro')
    ru=recall(y_test, p)
    ruj=recall(y_test, prr)
    py=precision(y_test, p)
    pyq=precision(y_test, prr)

    okl=oneError(y_test, p)
    rankl=rankingLoss(y_test, p)
    cokl=coverage(y_test, p)
    okl1=oneError(y_test, prr)
    rankl1=rankingLoss(y_test, prr)
    cokl1=coverage(y_test, prr)













    print("%s: %.5f%%" % ('subset Accuracy tpc', ac))
    x1.append(ac)
    print("%s: %.5f%%" % ('subset Accuracy', acq))
    x2.append(acq)

    print("%s: %.5f%%" % ('hamming Loss tpc', hm))
    x3.append(hm)
    print("%s: %.5f%%" % ('hamming Loss ', hmq))
    x4.append(hmq)

    print("%s: %.5f%%" % ('Accuracy tpc', acc))
    x5.append(acc)
    print("%s: %.5f%%" % ('Accuracy', accq))
    x6.append(accq)
    print("%s: %.5f%%" % ('precision tpc', py))
    x7.append(py)
    print("%s: %.5f%%" % ('precision', pyq))
    x8.append(pyq)

    print("%s: %.5f%%" % ('recall tpc', ru))
    x9.append(ru)
    print("%s: %.5f%%" % ('recall ', ruj))
    x10.append(ruj)

    print("%s: %.5f%%" % ('fbeta tpc', fb))
    x11.append(fb)
    print("%s: %.5f%%" % ('fbeta', fbq))
    x12.append(fbq)



    print("%s: %.5f%%" % ('accuracyMacro  tpc', am))
    x21.append(am)
    print("%s: %.5f%%" % ('accuracyMacro', amq))
    x22.append(amq)

    print("%s: %.5f%%" % ('accuracyMicro tpc', amm))
    x23.append(amm)
    print("%s: %.5f%%" % ('accuracyMicro', ammq))
    x24.append(ammq)





    print("%s: %.5f%%" % ('precisionMacro tpc', pm))
    x25.append(pm)
    print("%s: %.5f%%" % ('precisionMacro ', pmq))
    x26.append(pmq)

    print("%s: %.5f%%" % ('precisionMicro tpc', pr))
    x27.append(pr)
    print("%s: %.5f%%" % ('precisionMicro', prq))
    x28.append(prq)

    print("%s: %.5f%%" % ('recallMacro tpc', rm))
    x29.append(rm)
    print("%s: %.5f%%" % ('recallMacro', rmq))
    x30.append(rmq)

    print("%s: %.5f%%" % ('recallMicro tpc', rmm))
    x31.append(rmm)
    print("%s: %.5f%%" % ('recallMicro', rmmq))
    x32.append(rmmq)


    print("%s: %.5f%%" % ('fbetaMacro  tpc', fbm1))
    x33.append(fbm1)
    print("%s: %.5f%%" % ('fbetaMacro', fbm))
    x34.append(fbm)

    print("%s: %.5f%%" % ('fbetaMicro tpc', fbmm1))
    x35.append(fbmm1)
    print("%s: %.5f%%" % ('fbetaMicro', fbmm))
    x36.append(fbmm)

    print("%s: %.5f%%" % ('roc_auc macro  tpc', aucc2))
    x37.append(aucc2)
    print("%s: %.5f%%" % ('roc_auc macro', aucc22))
    x38.append(aucc22)

    print("%s: %.5f%%" % ('roc_auc micro tpc', aucc1))
    x39.append(aucc1)
    print("%s: %.5f%%" % ('roc_auc micro', aucc11))
    x40.append(aucc11)

    print("%s: %.5f%%" % ('oneError tpc', okl))
    x13.append(okl)
    print("%s: %.5f%%" % ('oneError', okl1))
    x14.append(okl1)

    print("%s: %.5f%%" % ('rankingLoss tpc', rankl))
    x15.append(rankl)
    print("%s: %.5f%%" % ('rankingLoss ', rankl1))
    x16.append(rankl1)

    print("%s: %.5f%%" % ('coverage tpc', cokl))
    x17.append(cokl)
    print("%s: %.5f%%" % ('coverage', cokl1))
    x18.append(cokl1)
    seconds = time.time() - start_time
    print('Time Taken:', time.strftime("%H:%M:%S",time.gmtime(seconds)))

  dx1.append(np.mean(x1))
  dx2.append(np.mean(x2))
  dx3.append(np.mean(x3))
  dx4.append(np.mean(x4))
  dx5.append(np.mean(x5))
  dx6.append(np.mean(x6))
  dx7.append(np.mean(x7))
  dx8.append(np.mean(x8))
  dx9.append(np.mean(x9))
  dx10.append(np.mean(x10))
  dx11.append(np.mean(x11))
  dx12.append(np.mean(x12))
  dx13.append(np.mean(x13))
  dx14.append(np.mean(x14))
  dx15.append(np.mean(x15))
  dx16.append(np.mean(x16))
  dx17.append(np.mean(x17))
  dx18.append(np.mean(x18))
  dx21.append(np.mean(x21))
  dx22.append(np.mean(x22))
  dx23.append(np.mean(x23))
  dx24.append(np.mean(x24))
  dx25.append(np.mean(x25))
  dx26.append(np.mean(x26))
  dx27.append(np.mean(x27))
  dx28.append(np.mean(x28))
  dx29.append(np.mean(x29))
  dx30.append(np.mean(x30))
  dx31.append(np.mean(x31))
  dx32.append(np.mean(x32))
  dx33.append(np.mean(x33))
  dx34.append(np.mean(x34))
  dx35.append(np.mean(x35))
  dx36.append(np.mean(x36))
  dx37.append(np.mean(x37))
  dx38.append(np.mean(x38))
  dx39.append(np.mean(x39))
  dx40.append(np.mean(x40))
  km+=1
se = time.time() - st
print('Time Taken all :', time.strftime("%H:%M:%S",time.gmtime(se)))


print("**********************************")
print("**********************************")
print('subset Accuracy tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(dx1)*100, np.std(dx1)*100))
print('subset Accuracy',"%.5f%% (+/- %.5f%%)" % (np.mean(dx2)*100, np.std(dx2)*100))

print('hamming Loss tpc',"%.5f (+/- %.5f)" % (np.mean(dx3), np.std(dx3)))
print('hamming Loss',"%.5f (+/- %.5f)" % (np.mean(dx4), np.std(dx4)))

print('Accuracy tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(dx5)*100, np.std(dx5)*100))
print('Accuracy',"%.5f%% (+/- %.5f%%)" % (np.mean(dx6)*100, np.std(dx6)*100))

print('precision  tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(dx7)*100, np.std(dx7)*100))
print('precision ',"%.5f%% (+/- %.5f%%)" % (np.mean(dx8)*100, np.std(dx8)*100))

print('recall tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(dx9)*100, np.std(dx9)*100))
print('recall',"%.5f%% (+/- %.5f%%)" % (np.mean(dx10)*100, np.std(dx10)*100))

print('fbeta tpc',"%.5f (+/- %.5f)" % (np.mean(dx11)*100, np.std(dx11)))
print('fbeta',"%.5f (+/- %.5f)" % (np.mean(dx12)*100, np.std(dx12)))


print('accuracyMacro tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(dx21)*100, np.std(dx21)*100))
print('accuracyMacro',"%.5f%% (+/- %.5f%%)" % (np.mean(dx22)*100, np.std(dx22)*100))

print('accuracyMicro tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(dx23)*100, np.std(dx23)*100))
print('accuracyMicro',"%.5f%% (+/- %.5f%%)" % (np.mean(dx24)*100, np.std(dx24)*100))

print('precisionMacro tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(dx25)*100, np.std(dx25)*100))
print('precisionMacro',"%.5f%% (+/- %.5f%%)" % (np.mean(dx26)*100, np.std(dx26)*100))

print('precisionMicro tpc',"%.5f (+/- %.5f)" % (np.mean(dx27)*100, np.std(dx27)*100))
print('precisionMicro',"%.5f (+/- %.5f)" % (np.mean(dx28)*100, np.std(dx28)*100))

print('recallMacro tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(dx29)*100, np.std(dx29)*100))
print('recallMacro',"%.5f%% (+/- %.5f%%)" % (np.mean(dx30)*100, np.std(dx30)*100))

print('recallMicro tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(dx31)*100, np.std(dx31)*100))
print('recallMicro',"%.5f%% (+/- %.5f%%)" % (np.mean(dx32)*100, np.std(dx32)*100))

print('fbetaMacro tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(dx33)*100, np.std(dx33)*100))
print('fbetaMacro',"%.5f%% (+/- %.5f%%)" % (np.mean(dx34)*100, np.std(dx34)*100))

print('fbetaMicro tpc',"%.5f (+/- %.5f)" % (np.mean(dx35)*100, np.std(dx35)))
print('fbetaMicro',"%.5f (+/- %.5f)" % (np.mean(dx36)*100, np.std(dx36)))

print('roc_auc macro tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(dx37)*100, np.std(dx37)*100))
print('roc_auc macro',"%.5f%% (+/- %.5f%%)" % (np.mean(dx38)*100, np.std(dx38)*100))

print('roc_auc micro tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(dx39)*100, np.std(dx39)*100))
print('roc_auc micro',"%.5f%% (+/- %.5f%%)" % (np.mean(dx40)*100, np.std(dx40)*100))
print('oneError tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(dx13), np.std(dx13)))
print('oneError',"%.5f%% (+/- %.5f%%)" % (np.mean(dx14), np.std(dx14)))

print('rankingLoss tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(dx15), np.std(dx15)))
print('rankingLoss',"%.5f%% (+/- %.5f%%)" % (np.mean(dx16), np.std(dx16)))

print('coverage tpc',"%.5f%% (+/- %.5f%%)" % (np.mean(dx17), np.std(dx17)))
print('coverage',"%.5f%% (+/- %.5f%%)" % (np.mean(dx18), np.std(dx18)))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
**********************************
Training on Fold:  1  result: 1
**********************************
**********************************


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


subset Accuracy tpc: 0.59000%
subset Accuracy: 0.57500%
hamming Loss tpc: 0.02607%
hamming Loss : 0.02643%
Accuracy tpc: 0.71771%
Accuracy: 0.71479%
precision tpc: 0.77750%
precision: 0.77000%
recall tpc: 0.75850%
recall : 0.77100%
fbeta tpc: 0.76788%
fbeta: 0.77050%
accuracyMacro  tpc: 0.97393%
accuracyMacro: 0.97357%
accuracyMicro tpc: 0.97393%
accuracyMicro: 0.97357%
precisionMacro tpc: 0.47744%
precisionMacro : 0.57052%
precisionMicro tpc: 0.71192%
precisionMicro: 0.77305%
recallMacro tpc: 0.58000%
recallMacro: 0.48605%
recallMicro tpc: 0.78467%
recallMicro: 0.72185%
fbetaMacro  tpc: 0.49562%
fbetaMacro: 0.49832%
fbetaMicro tpc: 0.74653%
fbetaMicro: 0.74658%
roc_auc macro  tpc: 0.73165%
roc_auc macro: 0.73549%
roc_auc micro tpc: 0.85039%
roc_auc micro: 0.85489%
oneError tpc: 0.97000%
oneError: 0.97500%
rankingLoss tpc: 0.24564%
rankingLoss : 0.23451%
coverage tpc: 21.52000%
coverage: 21.31500%
Time Taken: 00:00:26
**********************************
Training on Fold:  2  result: 1
*

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


subset Accuracy tpc: 0.55500%
subset Accuracy: 0.54000%
hamming Loss tpc: 0.02643%
hamming Loss : 0.02643%
Accuracy tpc: 0.68980%
Accuracy: 0.69680%
precision tpc: 0.76250%
precision: 0.76667%
recall tpc: 0.72933%
recall : 0.76183%
fbeta tpc: 0.74555%
fbeta: 0.76424%
accuracyMacro  tpc: 0.97357%
accuracyMacro: 0.97357%
accuracyMicro tpc: 0.97357%
accuracyMicro: 0.97357%
precisionMacro tpc: 0.50850%
precisionMacro : 0.63392%
precisionMicro tpc: 0.69536%
precisionMicro: 0.77500%
recallMacro tpc: 0.64301%
recallMacro: 0.52369%
recallMicro tpc: 0.78947%
recallMicro: 0.71854%
fbetaMacro  tpc: 0.52279%
fbetaMacro: 0.52669%
fbetaMicro tpc: 0.73944%
fbetaMicro: 0.74570%
roc_auc macro  tpc: 0.74769%
roc_auc macro: 0.75464%
roc_auc micro tpc: 0.84240%
roc_auc micro: 0.85333%
oneError tpc: 0.97500%
oneError: 0.97500%
rankingLoss tpc: 0.27443%
rankingLoss : 0.24421%
coverage tpc: 21.22000%
coverage: 21.18000%
Time Taken: 00:00:14
**********************************
Training on Fold:  3  result: 1
*

...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


subset Accuracy tpc: 0.59000%
subset Accuracy: 0.58500%
hamming Loss tpc: 0.02625%
hamming Loss : 0.02607%
Accuracy tpc: 0.72155%
Accuracy: 0.72738%
precision tpc: 0.77875%
precision: 0.78042%
recall tpc: 0.77725%
recall : 0.79725%
fbeta tpc: 0.77800%
fbeta: 0.78874%
accuracyMacro  tpc: 0.97375%
accuracyMacro: 0.97393%
accuracyMicro tpc: 0.97375%
accuracyMicro: 0.97393%
precisionMacro tpc: 0.51307%
precisionMacro : 0.61849%
precisionMicro tpc: 0.73510%
precisionMicro: 0.76174%
recallMacro tpc: 0.62862%
recallMacro: 0.53457%
recallMicro tpc: 0.76817%
recallMicro: 0.75166%
fbetaMacro  tpc: 0.51995%
fbetaMacro: 0.53794%
fbetaMicro tpc: 0.75127%
fbetaMicro: 0.75667%
roc_auc macro  tpc: 0.74875%
roc_auc macro: 0.75909%
roc_auc micro tpc: 0.86123%
roc_auc micro: 0.86913%
oneError tpc: 0.97500%
oneError: 0.97500%
rankingLoss tpc: 0.22798%
rankingLoss : 0.20924%
coverage tpc: 21.61500%
coverage: 21.74000%
Time Taken: 00:00:14
**********************************
Training on Fold:  49  result: 10

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


subset Accuracy tpc: 0.59500%
subset Accuracy: 0.57500%
hamming Loss tpc: 0.02589%
hamming Loss : 0.02571%
Accuracy tpc: 0.71200%
Accuracy: 0.71533%
precision tpc: 0.76667%
precision: 0.77042%
recall tpc: 0.74958%
recall : 0.77208%
fbeta tpc: 0.75803%
fbeta: 0.77125%
accuracyMacro  tpc: 0.97411%
accuracyMacro: 0.97429%
accuracyMicro tpc: 0.97411%
accuracyMicro: 0.97429%
precisionMacro tpc: 0.55101%
precisionMacro : 0.68395%
precisionMicro tpc: 0.71854%
precisionMicro: 0.77622%
recallMacro tpc: 0.70595%
recallMacro: 0.56323%
recallMicro tpc: 0.78339%
recallMicro: 0.73510%
fbetaMacro  tpc: 0.56959%
fbetaMacro: 0.56807%
fbetaMicro tpc: 0.74957%
fbetaMicro: 0.75510%
roc_auc macro  tpc: 0.76835%
roc_auc macro: 0.77406%
roc_auc micro tpc: 0.85361%
roc_auc micro: 0.86151%
oneError tpc: 0.97500%
oneError: 0.97500%
rankingLoss tpc: 0.25414%
rankingLoss : 0.23334%
coverage tpc: 21.12500%
coverage: 21.14000%
Time Taken: 00:00:14
**********************************
Training on Fold:  50  result: 10

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
